In [46]:
import pandas as pd
import numpy as np

In [47]:
df = pd.read_excel('C:/Users/ScMa9004/Documents/Panel/NEW PANEL/NEW_PANEL_RUSSIA_314.xlsx')

In [48]:
arc = pd.read_excel('C:/Users/ScMa9004/Documents/All Russia cities 01.01.2019_ for_2020.xlsx',
                    sheet_name = 'All_Russia_cities', 
                    skiprows = 1)

In [49]:
panrep = pd.read_excel('C:/Users/ScMa9004/Documents/Итоги визита/315/panel_report_00001_30.01.2020.xlsx', 
                       sheet_name = 'panel_report_00001_30.01.2020')

In [50]:
uo200 = pd.read_excel('C:/Users/ScMa9004/Documents/Universe Over 200.xls', sheet_name = 'Towns&Villages', skiprows = 1)

In [51]:
rules = pd.read_excel('C:/Users/ScMa9004/Documents/Panel/NEW PANEL/FMCG_CIG_BEER.xlsx',
                      sheet_name = 'index + mcellname')

In [52]:
# нужно было один раз
#merge L20 & M20
for i in df['ccode'][(df.hip == 'L20') | (df.hip == 'M20')].unique():
    df.loc[(df.ccode == i) & ((df.hip == 'L20') | (df.hip == 'M20')), 'AP_FMCG'] = df['AP_FMCG'][(df.ccode == i) & ((df.hip == 'L20') | (df.hip == 'M20'))].sum()
    df.loc[(df.ccode == i) & ((df.hip == 'L20') | (df.hip == 'M20')), 'OP_FMCG'] = df['OP_FMCG'][(df.ccode == i) & ((df.hip == 'L20') | (df.hip == 'M20'))].sum()
    df.loc[(df.ccode == i) & ((df.hip == 'L20') | (df.hip == 'M20')), 'hip'] = np.nan
    
#тоже нужно один раз
lst = ['FMCG', 'CIG', 'BEER']
for k in lst:
    for i in df['ccode'][(df.hip == 'L') | (df.hip == 'M') | (df.hip == 'S')].unique():
        df.loc[(df.ccode == i) & ((df.hip == 'L') | (df.hip == 'M') | (df.hip == 'S')), 'AP_' + k] = df['AP_' + k][(df.ccode == i) & ((df.hip == 'L') | (df.hip == 'M') | (df.hip == 'S'))].sum()
        df.loc[(df.ccode == i) & ((df.hip == 'L') | (df.hip == 'M') | (df.hip == 'S')), 'OP_' + k] = df['OP_' + k][(df.ccode == i) & ((df.hip == 'L') | (df.hip == 'M') | (df.hip == 'S'))].sum()
        df.loc[(df.ccode == i) & ((df.hip == 'L') | (df.hip == 'M') | (df.hip == 'S')), 'hip'] = 'L'
        
df = df.drop_duplicates()
df.loc[df['hip'] == 'L', 'hip'] = np.nan

#нужно было один раз
df = pd.merge(df, arc[['City Code', 'City Name']].rename(columns = {'City Code': 'ccode'}), on = 'ccode', how = 'left')

In [53]:
def parser(mcell):
    if mcell.split('-')[0] in ['CNT', 'FST', 'VOL', 'URL', 'NWT', 'STH', 'MSO', 'SIB']:
        area = [mcell.split('-')[0]]
        if 'RR' in mcell.split('-')[-1]:
            ccodes = []
            subreg = list(map(int, mcell.split('-')[1].split('|')))
            types = ['OSU', 'OLA', 'OME', 'OSM', 'OPV', 'OMM', 'OMS']
            pgr = list(mcell.split('-')[2].split('|'))
            ds = list(mcell.split('-')[3].split('|'))
            hip = []
            ndx = [mcell.split('-')[-1]]
            NO_city = []
            ttr = ['MT|TT']
        else:
            ccodes = []
            subreg = list(map(int, mcell.split('-')[1].split('|')))
            if ('IML' in mcell.split('-')[2]) | ('IMS' in mcell.split('-')[2]):
                types = ['OKS', 'ONS', 'OPV', 'OSM']
                hip = [mcell.split('-')[2]]
            else:
                types = list(mcell.split('-')[2].split('|'))
                hip = []
            pgr = list(map(int, mcell.split('-')[3].split('|')))
            ttr = list(mcell.split('-')[4].split('|'))
            try:
                NO_city = list(map(int, mcell.split('-')[5].replace('(', '').replace(')', '').split('&')))
            except:
                NO_city = []
            ndx = [mcell.split('-')[-1]]
            ds = []
    else:
        if '|' in mcell.split('-')[0]:
            area = [mcell.split('-')[0] + ' ' + mcell.split('-')[1]]
        else:
            area = [mcell.split('-')[1]]
        ccodes = list(map(int, mcell.split('-')[0].split('|')))
        subreg = []
        hip = []
        if ('IML' in mcell.split('-')[2]) | ('IMS' in mcell.split('-')[2]):
            types = ['OKS', 'ONS', 'OPV', 'OSM']
            hip = [mcell.split('-')[2]]
        else:
            types = list(mcell.split('-')[2].split('|'))
        if ('Y' in mcell.split('-')[3]) | ('N' in mcell.split('-')[3]):
            ttr = ['TT']
            hip = list(mcell.split('-')[3])
        else:
            ttr = list(mcell.split('-')[3].split('|'))
        NO_city = []
        pgr = []
        ndx = [mcell.split('-')[-1]]
        ds = []
    return(area, ccodes, subreg, types, pgr, ttr, NO_city, hip, ds, ndx)

# panel report new mcells

In [54]:
def add_mcells_to_panrep(df, rules, arc, uo200):
    # prepare file

    wrk1 = arc[['City Code',
                '# of Federal Subject', 
                'pgr1']].rename(columns = {'City Code': 'citycode',
                                                                   '# of Federal Subject': 'sub'})


    wrk2 = uo200[['ACN NN',
                  'Federal Subject Number',
                  'pgr_MT', 
                  'Distance Strata']][uo200['ACN NN'].notna() &\
                                      (uo200['ACN NN'] != 'Не использовать')].rename(columns = {'ACN NN': 'citycode',
                                                                                                'Federal Subject Number': 'sub', 
                                                                                                'pgr_MT': 'pgr1'})
    wrk2 = wrk2[wrk2['citycode'] >= 10000]

    wrk = wrk1.append(wrk2, sort = False).drop_duplicates()

    df = df[df['allstat'] != 'P']
    df.loc[df['rum'].isna() & df['rux'].notna(), 'rum'] = df['rux']

    df = pd.merge(df, wrk, on = 'citycode', how = 'left')

    df.loc[df['mt'] == False, 'mt'] = 'TT'
    df.loc[df['mt'] == True, 'mt'] = 'MT'

    df.loc[df['hip'] == '1', 'hip'] = 'Y'
    df.loc[df['hip'] == '0', 'hip'] = 'N'

    df = df[['outlet10', 'rub', 'ruc', 'rum', 'type', 'bst', 'citycode', 'mt', 'hip', 'sub', 'pgr1', 'Distance Strata', 'ops']]


    df['fmcg'] = np.nan
    df['cig'] = np.nan
    df['beer'] = np.nan
    df['exxon'] = np.nan
    df['rrc'] = np.nan
    df['rrb'] = np.nan
    
    # mcell id for panel stores
    for i, j in rules.iterrows():
        conditions = parser(j['new_mcell_name'])
        if conditions[1]:
            c1 = df['citycode'].isin(conditions[1])
        else:
            c1 = df['citycode'].isin(df['citycode'])
        if conditions[2]:
            c2 = df['sub'].isin(conditions[2])
        else:
            c2 = df['sub'].isin(df['sub'])
        if conditions[3]:
            c3 = df['type'].isin(conditions[3])
        else:
            c3 = df['type'].isin(df['type'])
        if conditions[4]:
            c4 = df['pgr1'].isin(conditions[4])
        else:
            c4 = df['pgr1'].isin(df['pgr1'])
        if conditions[5]:
            if '|' not in conditions[5][0]:
                c5 = df['mt'].isin(conditions[5])
            else:
                c5 = df['mt'].isin(df['mt'])
        else:
            c5 = df['mt'].isin(df['mt'])
        if conditions[6]:
            c6 = ~df['citycode'].isin(conditions[6])
        else:
            c6 = df['citycode'].isin(df['citycode'])
        if conditions[7]:
            c7 = df['hip'].isin(conditions[7])
        else:
            c7 = df['hip'].isin(df['hip'])
        if conditions[8]:
            c8 = df['Distance Strata'].isin(conditions[8])
        else:
            c8 = df['Distance Strata'].isin(df['Distance Strata'])
        if conditions[9][0] == 'FMCG':
            c9 = df['rum'].notna()
            c10 = (df['citycode'] < 10000)
            df.loc[c1 & c2 & c3 & c4 & c5 & c6 & c7 & c8 & c9 & c10, 'fmcg'] = j['index']
        elif conditions[9][0] == 'CIG':
            c9 = df['ruc'].notna()
            c10 = (df['citycode'] < 10000)
            df.loc[c1 & c2 & c3 & c4 & c5 & c6 & c7 & c8 & c9 & c10, 'cig'] = j['index']
        elif conditions[9][0] == 'BEER':
            c9 = df['rub'].notna()
            c10 = (df['citycode'] < 10000)
            df.loc[c1 & c2 & c3 & c4 & c5 & c6 & c7 & c8 & c9 & c10, 'beer'] = j['index']
        elif conditions[9][0] == 'EXXON':
            c9 = df['rum'].notna()
            c10 = (df['citycode'] < 10000)
            df.loc[c1 & c2 & c3 & c4 & c5 & c6 & c7 & c8 & c9 & c10, 'exxon'] = j['index']
        elif conditions[9][0] == 'RRC':
            c9 = df['ruc'].notna()
            c10 = (df['citycode'] >= 10000)
            df.loc[c1 & c2 & c3 & c4 & c5 & c6 & c7 & c8 & c9 & c10, 'rrc'] = j['index']
        elif conditions[9][0] == 'RRB':
            c9 = df['rub'].notna()
            c10 = (df['citycode'] >= 10000)
            df.loc[c1 & c2 & c3 & c4 & c5 & c6 & c7 & c8 & c9 & c10, 'rrb'] = j['index']
    
    #df.loc[(df.hip == 'L20') | (df.hip == 'M20') | (df.hip == 'L') | (df.hip == 'M') | (df.hip == 'S'), 'hip'] = np.nan
    
    if len(df[df.fmcg.isna() & df.cig.isna() & df.beer.isna() & df.rrc.isna() & df.rrb.isna() & df.exxon.isna()]) > 0:
        print('Эти тт не попали ни в один микроселл:')
        print(df['outlet10'][df.fmcg.isna() & df.cig.isna() & df.beer.isna() & df.rrc.isna() & df.rrb.isna() & df.exxon.isna()])
    
    
    return df

In [55]:
panrep = add_mcells_to_panrep(panrep, rules, arc, uo200)

Эти тт не попали ни в один микроселл:
1603    2010001854
1929    2010002183
8788    2000MC7148
Name: outlet10, dtype: object


In [56]:
panrep.loc[panrep['hip'].isin(['S', 'M', 'L', 'M20', 'L20']), 'hip'] = np.nan

In [60]:
# see stores without mcells
panrep[panrep['outlet10'] == '2000MC7148']

,outlet10,rub,ruc,rum,type,bst,citycode,mt,hip,sub,pgr1,Distance Strata,ops,fmcg,cig,beer,exxon,rrc,rrb
8788,2000MC7148,NaN,NaN,A,OOP,1,589,TT,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
#если необходимо - определить микроселл для нераскидавшихся тт
panrep.loc[panrep['outlet10'] == '2010001854', 'cig'] = 5543
panrep.loc[panrep['outlet10'] == '2010002183', 'fmcg'] = 1334

In [367]:
panrep.to_excel('panel_report-new_mcells.xlsx')

# NDX

In [80]:
ndx = pd.DataFrame(columns = {'outlet10', 'index'})
for i in ['fmcg', 'cig', 'beer', 'exxon', 'rrc', 'rrb']:
    ndx = ndx.append(panrep[['outlet10', i]].rename(columns = {i: 'index'}).dropna())

ndx = pd.merge(ndx, rules[['index', 'CELL_ID']], on = 'index', how = 'left')

ndx.to_excel('ndx.xlsx', index = False)

# new_panel_op_ap

In [368]:
a = pd.pivot_table(panrep, index = 'fmcg', values = 'outlet10', aggfunc = 'count').reset_index().rename(columns = {'fmcg': 'index'})
b = pd.pivot_table(panrep, index = 'cig', values = 'outlet10', aggfunc = 'count').reset_index().rename(columns = {'cig': 'index'})
c = pd.pivot_table(panrep, index = 'beer', values = 'outlet10', aggfunc = 'count').reset_index().rename(columns = {'beer': 'index'})
d = pd.pivot_table(panrep, index = 'exxon', values = 'outlet10', aggfunc = 'count').reset_index().rename(columns = {'exxon': 'index'})
e = pd.pivot_table(panrep, index = 'rrc', values = 'outlet10', aggfunc = 'count').reset_index().rename(columns = {'rrc': 'index'})
f = pd.pivot_table(panrep, index = 'rrb', values = 'outlet10', aggfunc = 'count').reset_index().rename(columns = {'rrb': 'index'})
res = a.append(b).append(c).append(c).append(d).append(e).append(f).rename(columns = {'outlet10': 'AP'})

res = pd.merge(rules, res, on = 'index', how = 'left').drop_duplicates()

res['AP'] = res['AP'].fillna(0)

res['ndx'] = ''
res.loc[res['index'] < 4000, 'ndx'] = 'fmcg'
res.loc[(res['index'] >= 4000) & (res['index'] < 6000), 'ndx'] = 'cig'
res.loc[(res['index'] >= 6000) & (res['index'] < 7000), 'ndx'] = 'beer'
res.loc[(res['index'] >= 7000) & (res['index'] < 8000), 'ndx'] = 'exxon'
res.loc[(res['index'] >= 10000) & (res['index'] < 11000), 'ndx'] = 'rrc'
res.loc[res['index'] >= 11000, 'ndx'] = 'rrb'

res.to_excel('new_panel_op_ap.xlsx')

# panel russia

In [369]:
#create columns for new AP and fill

df['fmcg'] = 0
df['cig'] = 0
df['beer'] = 0
df['exxon'] = 0

for k in ['fmcg', 'cig', 'beer', 'exxon']:
    for i, j in panrep[panrep[k].notna()].iterrows():
        ccode = df['ccode'] == j['citycode']
        stype = df['type'] == j['type']
        ttr = df['ttr'] == j['mt']
        mcell = df['MCELL_ID_' + k.upper()] == j[k]
        if isinstance(j['hip'], str):
            hip = df['hip'] == j['hip']
        else:
            hip = df['hip'].isna()
            
        if len(df[k][mcell & ccode & stype & ttr & hip]) > 1:
            print('Эти строки дублируют друг друга',df[mcell & ccode & stype & ttr & hip])
            
        elif len(df[k][mcell & ccode & stype & ttr & hip]) == 0:
            if len(df[k][ccode & stype & ttr & hip]) == 1:
                #if row has wrong mcell (after new panel russia implementation)
                df.loc[df.index[ccode & stype & ttr & hip], 'MCELL_ID_' + k.upper()] = j[k]
                ccode = df['ccode'] == j['citycode']
                stype = df['type'] == j['type']
                ttr = df['ttr'] == j['mt']
                mcell = df['MCELL_ID_' + k.upper()] == j[k]
                if isinstance(j['hip'], str):
                    hip = df['hip'] == j['hip']
                else:
                    hip = df['hip'].isna()
            else:
                #show store which can not find row in panel russia
                n = len(df) + 1
                df.loc[n, 'ccode'] = j['citycode']
                df.loc[n, 'type'] = j['type']
                df.loc[n, 'ttr'] = j['mt']
                df.loc[n, 'MCELL_ID_' + k.upper()] = j[k]
                df.loc[n, 'hip'] = j['hip']
                df.loc[n, 'Sub_id'] = j['sub']
                df.loc[n, 'pgr1'] = j['pgr1']
                df.loc[n, 'ops'] = j['ops']
                df.loc[n, 'fmcg'] = 0
                df.loc[n, 'cig'] = 0
                df.loc[n, 'beer'] = 0
                df.loc[n, 'exxon'] = 0
                ccode = df['ccode'] == j['citycode']
                stype = df['type'] == j['type']
                ttr = df['ttr'] == j['mt']
                mcell = df['MCELL_ID_' + k.upper()] == j[k]
                if isinstance(j['hip'], str):
                    hip = df['hip'] == j['hip']
                else:
                    hip = df['hip'].isna()
                print(i, j['type'], j['citycode'], j['mt'], j[k], k)
        df.loc[mcell & ccode & stype & ttr & hip, k] = df[k][mcell & ccode & stype & ttr & hip] + 1

1717 OMS 555 TT 1922.0 fmcg
1767 OPR 583 TT 1840.0 fmcg
1771 FKT 541 TT 3111.0 fmcg
1802 OBS 1463 MT 1207.0 fmcg
1808 FKF 1004 TT 3148.0 fmcg
1851 OWS 1014 TT 1285.0 fmcg
1924 OPV 1473 TT 2683.0 fmcg
1929 OHS 1040 MT 1334.0 fmcg
1603 FKT 973 TT 5543.0 cig
1758 OME 940 TT 4835.0 cig
1775 OKT 193 TT 4311.0 cig
1796 OTK 348 TT 5545.0 cig
1878 OMM 345 MT 4245.0 cig
1900 OWS 963 TT 4839.0 cig
1948 OME 467 TT 4313.0 cig
1989 OME 1605 TT 4805.0 cig
1867 OSM 1516 TT 6390.0 beer


In [374]:
#сравнение старой  и новой АП
df[['fmcg', 'AP_FMCG', 'cig', 'AP_CIG', 'beer', 'AP_BEER', 'exxon', 'AP_EXXON']].sum()

fmcg        7159.0
AP_FMCG     7235.0
cig         9214.0
AP_CIG      9272.0
beer        4591.0
AP_BEER     4605.0
exxon        278.0
AP_EXXON     277.0
dtype: float64

In [375]:
#stores in indexes in panel report
print(len(panrep[panrep.fmcg.notna()]),
len(panrep[panrep.cig.notna()]),
len(panrep[panrep.beer.notna()]),
len(panrep[panrep.exxon.notna()]))

7159 9214 4591 278


In [372]:
#перераспределение внутри микроселлов
for k in ['fmcg', 'cig', 'beer', 'exxon']:
    for i, j in df[df['MCELL_ID_' + k.upper()].notna() & (df[k] > df['OP_' + k.upper()])].iterrows():
        #если в некоторой строке ап увеличилась и есть оверсэмпл и ОП всего микроселла >= АП всего микроселла
        if (j[k] > j['AP_' + k.upper()]) &\
        (j[k] > j['OP_' + k.upper()]) &\
        (df['OP_' + k.upper()][df['MCELL_ID_' + k.upper()] == j['MCELL_ID_' + k.upper()]].sum() >=\
        df[k][df['MCELL_ID_' + k.upper()] == j['MCELL_ID_' + k.upper()]].sum()):
            while (df[k][i] - df['OP_' + k.upper()][i]) > 0:
                for l in df.index[df['MCELL_ID_' + k.upper()] == j['MCELL_ID_' + k.upper()]]:
                    if df['OP_' + k.upper()][l] > df[k][l]:
                        df.loc[l, 'OP_' + k.upper()] = df['OP_' + k.upper()][l] - 1
                        df.loc[i, 'OP_' + k.upper()] = df['OP_' + k.upper()][i] + 1
                        if (df[k][i] - df['OP_' + k.upper()][i]) == 0:
                            break

In [376]:
df['AP_FMCG'] = df['fmcg']
df['AP_CIG'] = df['cig']
df['AP_BEER'] = df['beer']
df['AP_EXXON'] = df['exxon']

In [377]:
df = df.drop(columns = ['fmcg', 'cig', 'beer', 'exxon'])

In [378]:
df.to_excel('Panel_russia_315.xlsx')